## basics

- 参考
    - https://huggingface.co/blog/llama2
- 4k tokens of context
- 几个版本：3\*2\*2 == 12
    - https://huggingface.co/meta-llama
    - 7B，13B，70B
    - 是否是 hf
    - 是否是 chat
        - meta-llama/Llama-2-7b
        - meta-llama/Llama-2-7b-hf
        - meta-llama/Llama-2-7b-chat
        - meta-llama/Llama-2-7b-chat-hf

## text-generation

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from transformers import BitsAndBytesConfig
import torch

In [9]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False)}, clean_up_tokenization_spaces=False)

In [11]:
pipeline.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [13]:
trainable_params = 0
all_param = 0
for name, param in pipeline.model.named_parameters():
#     print(name, param.requires_grad, param.numel())
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(trainable_params, all_param, trainable_params*1./all_param)

6738415616 6738415616 1.0


In [14]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [15]:
for seq in sequences:
    print(seq['generated_text'] + '\n\n')

I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

I also enjoyed "The Sopranos" and "The Wire". Have you seen those shows?

I'm also interested in historical documentaries. Have you seen "The Civil War" or "The Roosevelts"?

Let me know if you have any other recommendations. Thanks!


I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

I'm a big fan of crime dramas and historical dramas. I also enjoy shows with complex characters and engaging storylines.

Thanks!

Answer: Based on your interest in "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Wire" - This crime drama explores the drug trade in Baltimore from multiple perspectives, featuring complex characters and a gripping storyline.

2. "Narcos" - This series follows the rise and fall of drug kingpin Pablo Escobar and the DEA agents tasked with bringing him down. It's a gripp

In [16]:
sequences = pipeline(
    '我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(seq['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？
Ћ�找到更多的电影推荐，我喜欢这样的电影类型。
我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？
Љ� says:
I am a big fan of movies, and I have watched many classic and contemporary films. Here are some of my favorite movie genres and why I like them:
Classic Movies: I enjoy watching old movies from the 1940s to the 1960s, such as "Casablanca," "The Godfather," and "Singin' in the Rain." These films have a timeless quality that transcends their historical context, and they offer a window into the cultural and social values of the time.
Action Movies: I like action movies because they are fast-paced and
我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？
Љ�陶�论坛
�际�哈达娱乐
�际�哈达娱乐
爱情�电影
�际�哈达娱乐
�际�哈达娱乐
�际�哈达娱乐
�际�哈达娱乐
�际�哈达娱乐
�际�哈达娱乐
�际���


## fine-tune

- trl
    - SFTTrainer

```
pip install trl
git clone https://github.com/lvwerra/trl
python trl/examples/scripts/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name timdettmers/openassistant-guanaco \
    --load_in_4bit \
    --use_peft \
    --batch_size 16 \
    --gradient_accumulation_steps 2
```

- 对于 4090，大概是需要 5个小时
    - 3 个 epochs
    - 16*2 的 batch size
    - (9846 * 3) / (16*2) == 924 个 optimizer steps

### guanaco dataset

In [17]:
from datasets import load_dataset

In [18]:
dataset = load_dataset('timdettmers/openassistant-guanaco', split='train')

Found cached dataset json (/media/whaow/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-c93588435bc90172/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [19]:
dataset

Dataset({
    features: ['text'],
    num_rows: 9846
})

In [20]:
print(dataset[0]['text'])

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading

In [21]:
for part in dataset[0]['text'].split('###'):
    print(part)


 Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.
 Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to 

In [ ]:
# optimization steps
# len(dataset) * epochs / (batch_size * gradient_accumulation_steps * n_gpus)
9846*3/(16*2*1)